In [3]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import pandas as pd
from tqdm.notebook import tqdm
from pretty_midi import PrettyMIDI

from src.db import reference_sets as db
from src.utils import strings
from src.evaluation import mgeval

In [2]:
# prepare database
db.create_tables()
set_id = db.store_ref_set('theorytab', 17752, 4, 'https://github.com/wayne391/symbolic-musical-datasets, melodies crawled from https://www.hooktheory.com/theorytab')

In [3]:
# batch evaluation of all reference midis + store results in db (only similarity distances or also absolute values?)

set_name = 'theorytab' 

source_folder = '../data/reference_data/' + set_name
i = 0
for root, dirs, files in tqdm(os.walk(source_folder)):
    for file in files:
        if "call" in file and "log" not in file: 
            pair_number = file[0:2]
            call_file = root + "/" + file
            response_file = root + "/" + pair_number + "_response.mid"
            pm = PrettyMIDI(call_file)
            call_analysis = mgeval.analyze_midi(call_file, 4)
            response_analysis = mgeval.analyze_midi(response_file, 4)
            similarity_distances = mgeval.calc_distances(call_analysis, response_analysis)

            song_name = strings.remove_prefix(root, source_folder + "\\")
            print(song_name)
            db.store_ref_data(set_id, song_name, pair_number, **similarity_distances)

0it [00:00, ?it/s]


z_zun_a-world-of-nightmares-never-seen-before_pre-chorus
z_zun_a-world-of-nightmares-never-seen-before_verse
z_zun_a-world-of-nightmares-never-seen-before_verse
z_zun_alice-in-wonderland_chorus
z_zun_alice-in-wonderland_intro
z_zun_bamboo-forest-of-the-full-moon_intro
z_zun_bamboo-forest-of-the-full-moon_pre-chorus
z_zun_bamboo-forest-of-the-full-moon_verse
z_zun_battlefield-of-hanahazama_chorus
z_zun_battlefield-of-hanahazama_chorus
z_zun_battlefield-of-hanahazama_intro
z_zun_battlefield-of-hanahazama_pre-chorus
z_zun_battlefield-of-hanahazama_pre-chorus
z_zun_battlefield-of-hanahazama_verse
z_zun_battlefield-of-hanahazama_verse
z_zun_because-princess-inada-is-scolding-me_intro
z_zun_because-princess-inada-is-scolding-me_verse
z_zun_because-princess-inada-is-scolding-me_verse
z_zun_bell-of-avici---infinite-nightmare_chorus
z_zun_bell-of-avici---infinite-nightmare_chorus
z_zun_bell-of-avici---infinite-nightmare_pre-chorus
z_zun_bell-of-avici---infinite-nightmare_verse
z_zun_bell-of-av

In [4]:
# fead reference data of data set 1 into a pandas dataframe
df = db.ref_data_table_to_dataframe(1)
df[0:5]

,id,set_id,song_name,pair_number,pitch_count_distance,pitch_count_per_bar_distance,pitch_class_histogram_distance,pitch_class_histogram_per_bar_distance,pitch_class_transition_matrix_distance,avg_pitch_interval_distance,pitch_range_distance,note_count_distance,note_count_per_bar_distance,note_length_histogram_distance,note_length_transition_matrix_distance,avg_ioi_distance
0,1,1,a_a-day-to-remember_downfall-of-us-all_intro-a...,1,2.0,2.236068,0.198554,0.198554,5.830952,0.608466,2.0,6.0,4.242641,0.139849,5.477226,0.082504
1,2,1,a_a-day-to-remember_downfall-of-us-all_pre-cho...,1,1.0,3.162278,0.511819,0.511819,4.242641,0.230769,5.0,3.0,5.000000,0.317655,6.000000,0.021110
2,3,1,a_a-day-to-remember_downfall-of-us-all_pre-cho...,2,0.0,2.236068,0.460652,0.460652,7.416198,0.092732,0.0,2.0,2.000000,0.095979,4.123106,0.035862
3,4,1,a_a-g-cook_superstar_chorus,1,1.0,2.000000,0.104842,0.104842,2.449490,0.041739,0.0,2.0,2.000000,0.091240,2.449490,0.026957
4,5,1,a_a-g-cook_superstar_pre-chorus-and-chorus,1,0.0,0.000000,0.002156,0.002156,1.414214,0.041481,0.0,2.0,2.000000,0.093808,3.464102,0.012165


In [ ]:
# calculate the average or q1, median, q3 + store it as reference for normalization
# => might be worth changing architecture to object orientation here (creating an instance of the evaluation class, where you can set certain reference parameters)